In [1]:
import os

In [2]:
%pwd

'c:\\testproj2\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\testproj2'

In [4]:
# Importing the 'dataclass' decorator from the 'dataclasses' module  
# This helps in creating a simple class to store data without writing boilerplate code  
from dataclasses import dataclass  

# Importing 'Path' from the 'pathlib' module  
# 'Path' is used for handling filesystem paths in an object-oriented way  
from pathlib import Path  

# Using the '@dataclass' decorator to automatically generate special methods  
# like '__init__', '__repr__', and '__eq__' for the class  
@dataclass  
class DataIngestionConfig:  
    # Root directory where all data-related files will be stored  
    root_dir: Path  
    # URL of the source data that needs to be downloaded  
    source_URL: str  
    # Local path where the downloaded data file will be stored  
    local_data_file: Path  
    # Directory where the extracted data will be stored after unzipping  
    unzip_dir: Path  


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [6]:
# Defining the ConfigurationManager class to manage configuration settings
class ConfigurationManager:  
    def __init__(self,  
                 config_filepath=CONFIG_FILE_PATH,  # Path to the main configuration file  
                 params_filepath=PARAMS_FILE_PATH,  # Path to the parameters file  
                 schema_filepath=SCHEMA_FILE_PATH):  # Path to the schema file  
        print(config_filepath)

        # Reading configuration settings from the YAML files  
        self.config = read_yaml(config_filepath)  # Loads the main configuration file  
        self.params = read_yaml(params_filepath)  # Loads the parameters file  
        self.schema = read_yaml(schema_filepath)  # Loads the schema file  

        # Creating necessary directories specified in the configuration  
        create_directories([self.config.artifacts_root])  # Ensures the artifact root directory exists  

    # Method to get the data ingestion configuration  
    def get_data_ingestion_config(self) -> DataIngestionConfig:  
        # Extracting data ingestion-related settings from the configuration  
        config = self.config.data_ingestion  

        # Creating the root directory for data ingestion if it doesn’t exist  
        create_directories([config.root_dir])  

        # Creating an instance of DataIngestionConfig with necessary parameters  
        data_ingestion_config = DataIngestionConfig(  
            root_dir=config.root_dir,  # Directory for storing ingested data  
            source_URL=config.source_URL,  # URL to fetch the data from  
            local_data_file=config.local_data_file,  # Path to store the downloaded data file  
            unzip_dir=config.unzip_dir  # Path where extracted files will be stored  
        )  

        # Returning the configured data ingestion object  
        return data_ingestion_config  


In [7]:
# Import necessary modules  
import os  # For interacting with the operating system  
import zipfile  # For handling ZIP files  
import urllib.request as request  # For downloading files from URLs  
from src.datascience import logger  # For logging messages 

In [8]:
# Component - Data Ingestion  

# Defining the DataIngestion class for handling data downloading and extraction  
class DataIngestion:  
    def __init__(self, config: DataIngestionConfig):  
        # Initializing with the data ingestion configuration  
        self.config = config  

    # Method to download the zip file from the source URL  
    def download_file(self):  
        # Checking if the file already exists in the local directory  
        if not os.path.exists(self.config.local_data_file):  
            # Downloading the file from the source URL and saving it locally  
            filename, headers = request.urlretrieve(  
                url=self.config.source_URL,  # URL to fetch the file  
                filename=self.config.local_data_file  # Path to save the downloaded file  
            )  
            # Logging the successful download and header information  
            logger.info(f"{filename} downloaded successfully! Details: \n{headers}")  
        else:  
            # Logging that the file already exists, avoiding redundant downloads  
            logger.info(f"File already exists at {self.config.local_data_file}")  

    # Method to extract the contents of the downloaded ZIP file  
    def extract_zip_file(self):  
        """  
        Extracts the ZIP file into the designated data directory.  
        """  
        unzip_path = self.config.unzip_dir  # Path to extract files  
        os.makedirs(unzip_path, exist_ok=True)  # Creating the extraction directory if it doesn’t exist  

        # Extracting the ZIP file using the zipfile module  
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:  
            zip_ref.extractall(unzip_path)  # Extracting all contents to the directory  

        # Logging the successful extraction  
        logger.info(f"Files extracted successfully to {unzip_path}")  


In [9]:
try:  
    # Creating an instance of ConfigurationManager to read configuration files  
    config = ConfigurationManager()  
    # Retrieving the data ingestion configuration settings  
    data_ingestion_config = config.get_data_ingestion_config()  
    # Creating an instance of DataIngestion with the retrieved configuration  
    data_ingestion = DataIngestion(config=data_ingestion_config)  
    # Downloading the data file from the source URL  
    data_ingestion.download_file()  
    # Extracting the contents of the downloaded ZIP file  
    data_ingestion.extract_zip_file()  

# Handling any exceptions that may occur during the process  
except Exception as e:  
    raise e  # Raising the exception to notify about errors  


config\config.yaml
[2025-02-06 22:19:05,975: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-06 22:19:05,977: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-06 22:19:05,980: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-06 22:19:05,981: INFO: common: created directory at: artifacts]
[2025-02-06 22:19:05,982: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-06 22:19:07,189: INFO: 2788082593: artifacts/data_ingestion/data.zip downloaded successfully! Details: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A056:4DE64:2734A2:369E20:67A4E807
Accept-Ranges